In [1]:
from IPython.display import Image
from pathlib import Path
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
import math
import xgboost as xgb

In [2]:
train_labels = pd.read_csv("KD/train_labels_KD.csv", index_col="seq_id")
train_labels_columns=train_labels.columns
train_labels['season'] = train_labels.index.map(lambda x: x.split("#")[0])


In [3]:
feature_classes=list(train_labels_columns)
feature_classes.remove('reptiles')
feature_classes.remove('caracal')
feature_classes.remove('wildcat')
feature_classes.remove('honeybadger')
feature_classes.remove('vulture')
feature_classes.remove('duiker')
feature_classes.remove('civet')
feature_classes.remove('genet')
feature_classes.remove('rhinoceros')
feature_classes.remove('zorilla')
feature_classes.remove('cattle')
feature_classes.remove('steenbok')
feature_classes.remove('bat')


In [4]:
everything_s10 = pd.read_csv("KD/everything_fullhd_SER_S10_add.csv", index_col="seq_id_buf")
everything_s10=everything_s10.iloc[:153319]
everything_s10.index.name='seq_id'
# everything_s10 = pd.read_csv("KD/everything_SER_S10.csv", index_col="seq_id_buf")
# everything_s10.index.name='seq_id'


In [5]:
# everything_S9 = pd.read_csv("KD/everything_fullhd_SER_S9.csv", index_col="seq_id")
# everything_S9 = pd.read_csv("everything_merge.csv", index_col="seq_id")
everything_S9 = pd.concat([pd.read_csv("KD/everything_fullhd_SER_S9.csv", index_col="seq_id"),  pd.read_csv("everything_merge.csv", index_col="seq_id")])

In [6]:
# everything_S9 = pd.read_csv("KD/everything_fullhd_SER_S9_add2.csv", index_col="seq_id_buf")
# everything_S9.index.name='seq_id'
# everything_S9_add= pd.read_csv("KD/everything_fullhd_SER_S9_add3.csv", index_col="seq_id_buf")
# everything_S9_add.index.name='seq_id'
# everything_S9_merge = pd.concat([everything_S9, everything_S9_add.iloc[94:]])
# everything_S9_merge.to_csv("KD/everything_fullhd_SER_S9.csv")

In [7]:
# everything_S9_add.iloc[94:]

In [8]:
len(everything_S9)


699208

In [9]:
# everything_s10['season'] = 10
# everything_S9['season'] = 9

In [10]:
# everything_s10_v2['v2']=True
# everything_s10['v2']=False
# everything_s10['v2']=False

In [11]:
# everything = pd.concat([everything_S9, everything_s10, everything_s10_v2])
everything = pd.concat([everything_S9,  everything_s10])


In [12]:
join = everything.join(train_labels, how='left')


In [13]:
len(join)

852527

In [14]:
len(join[join['season']=='SER_S9'])

699208

In [15]:
len(join[join['season']=='SER_S10'])

153319

In [16]:
models = [
            'pred_model_480_360_as512_img',
            'pred_model_480_360_as480_img',
            'pred_model_480_360_v2_as512_img', 
            'pred_model_480_360_v2_as480_img', 
            'pred_model_512_384_v2_as512_img'
         ]
models2 = [
           'pred_mean_back_2models_as512_',
           'pred_mean_back_2models_as480_',
          ]

models3 = [
           'pred_back_as480_',
           'pred_back_as512_',
          ]


In [17]:
interesting_columns = ['count_imgs', 'season']
models1 = []
for m in models:
    models1.append(m + '1_')
    models1.append(m + '2_')
    models1.append(m + '3_')
for m in models1 + models2 + models3 + [""]:
    for c in feature_classes:
        interesting_columns.append(m + c)
        

In [18]:
join=join[interesting_columns]

In [19]:
import numpy as np
lgbt_models = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird', 'hyenaspotted', 'lionfemale', 
'eland', 'hippopotamus', 'reedbuck', 'topi', 'baboon', 'dikdik', 
'secretarybird', 'jackal']

training_distibution = join.loc[train_labels.season.isin(['SER_S9']), lgbt_models].mean(axis=0)
# training_distibution = join.loc[train_labels.season.isin(['SER_S9', 'SER_S10']), lgbt_models].mean(axis=0)

test_distibution = train_labels[lgbt_models].mean(axis=0)

scale_pos_weight=test_distibution/training_distibution

for key, value in scale_pos_weight.items():
    # do something with value
    scale_pos_weight[key] = np.clip(value,0.6,1.8)
scale_pos_weight

empty              0.969667
wildebeest         1.147404
zebra              0.934102
gazellethomsons    0.932744
buffalo            1.635981
elephant           1.209374
hartebeest         1.800000
impala             1.684557
gazellegrants      1.698572
giraffe            1.385237
warthog            1.178625
guineafowl         1.800000
otherbird          1.243811
hyenaspotted       0.890472
lionfemale         1.302885
eland              1.800000
hippopotamus       0.936243
reedbuck           1.264991
topi               1.064517
baboon             0.944591
dikdik             1.136498
secretarybird      0.777513
jackal             0.640410
dtype: float64

In [20]:
# for model_id in ['pred_model_480_360_v2_as480_img1_', 'pred_model_480_360_v2_as480_img1_','pred_model_480_360_v2_as480_img1_',
#          'pred_model_480_360_v2_as512_img1_', 'pred_model_480_360_v2_as512_img2_','pred_model_480_360_v2_as512_img3_',
#          'pred_model_480_360_as480_img1_', 'pred_model_480_360_as480_img2_','pred_model_480_360_as480_img3_',
#          'pred_model_480_360_as512_img1_', 'pred_model_480_360_as512_img2_','pred_model_480_360_as512_img3_',
#           'pred_mean_back_2models_as512_','pred_back_as512_']:
#     join[model_id + 'nonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].sum(axis=1)
#     join[model_id + 'maxnonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].max(axis=1)

In [21]:
for c in feature_classes:
    print(c)
    join['median_img1_' + c] = join[[m + '1_' + c for m in models]].median(axis=1)
    join['median_img2_' + c]=join['median_img1_' + c]
    join['median_img3_' + c]=join['median_img1_' + c]
    join.loc[join.count_imgs > 1, 'median_img2_' + c] = join.loc[join.count_imgs > 1, [m + '2_' + c for m in models]].median(axis=1)
    join.loc[join.count_imgs > 2, 'median_img3_' + c] = join.loc[join.count_imgs > 2, [m + '3_' + c for m in models]].median(axis=1)
    join['mean_imgs_' + c] = (join['median_img1_' + c] + join['median_img2_' + c]*0.8 + join['median_img3_' + c]*0.5) /2.3
    
    join['mean_combines_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_combines_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models2]].median(axis=1)
    
    join['mean_backs_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_backs_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models3]].median(axis=1)
    

aardvark
aardwolf
baboon
batearedfox
buffalo
bushbuck
cheetah
dikdik
eland
elephant
empty
gazellegrants
gazellethomsons
giraffe
guineafowl
hare
hartebeest
hippopotamus
hyenaspotted
hyenastriped
impala
insectspider
jackal
koribustard
leopard
lionfemale
lionmale
mongoose
monkeyvervet
ostrich
otherbird
porcupine
reedbuck
rodents
secretarybird
serval
topi
warthog
waterbuck
wildebeest
zebra


In [22]:
DATA_PATH = Path("../../datasets/wildlife") 
train_metadata = pd.read_csv(DATA_PATH / "train_metadata.csv", parse_dates= ['datetime'], index_col="seq_id")
train_metadata = train_metadata.groupby('seq_id').first()
train_metadata['month']=train_metadata.datetime.dt.month
train_metadata['hour']=train_metadata.datetime.dt.hour
join = join.join(train_metadata.hour, how='left')
join = join.join(train_metadata.month, how='left')


In [23]:

join.loc[join.count_imgs<2, ['pred_back_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_back_as512_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as512_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes]]=-0.01

           



           

In [24]:

test_index = set(join[join.season=='SER_S10'].index[::2])
test_dataset     = join[join.index.isin(test_index)]
val_dataset = join[(join.season=='SER_S10') & (~join.index.isin(test_index))]
train_dataset = join[join.season=='SER_S9']
animal_columns =list(train_labels.columns)
animal_columns.remove('empty')



len(train_dataset), len(val_dataset), len(test_dataset)

(699208, 76659, 76660)

In [ ]:
from sklearn.metrics import log_loss
loss = 0
for c in feature_classes:
    x = test_dataset[['pred_model_480_360_v2_as512_img1_' + c, c]].values
    if sum(x[:,1]) !=0:
#         print(c)
        l = log_loss(x[:,1], x[:,0])
        loss+=l
        print(c, l)
print(loss/54)

In [26]:
import gc
del train_metadata
del everything_s10
del everything_S9
del train_labels
del join
gc.collect()

30

In [27]:


num_leaves={

}

max_depth={


}
lambda_l1={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}
lambda_l2={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}

learning_rate={
    'zebra':0.008,
    'empty': 0.008,
}

feature_fraction={
    'zebra':0.5,
}
bagging_fraction={
    'zebra':0.8,
    'empty':0.8, #?
}
bagging_freq={
    'zebra':5,
    'empty':5, #?
}
feature_fraction={
    'zebra':0.6,
    'empty':0.3,
}
# elephant
min_child_samples={
    'empty':30,
    'zebra':30,
    'buffalo':30,
    'topi':30,
    'wildebeest':30,
    'gazellethomsons':30
}

clazz='empty'

columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']


# columns = ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
#     ['mean_backs_' + c for c in feature_classes] +\
#     ['median_img1_' + c for c in feature_classes] +\
#     ['median_img2_' + c for c in feature_classes] +\
#     ['median_img3_' + c for c in feature_classes] +\
#     ['mean_combines_' + c for c in feature_classes] +\
#     ['hour', 'month']

In [28]:


# X_train = pd.concat([train_dataset[columns], test_dataset[columns]])
# y_train = pd.concat([train_dataset[clazz], test_dataset[clazz]])
X_train = train_dataset[columns]
y_train = train_dataset[clazz]
X_val = val_dataset[columns]
y_val = val_dataset[clazz]

In [29]:
len(X_train)

699208

In [30]:
clazz

'empty'

In [31]:
y_train.mean()


0.7836208967860779

In [32]:
y_val.mean()/y_train.mean()


1.0223134692577944

In [33]:
scale_pos_weight['warthog']

1.178624651309858

In [35]:


lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

scale = y_train.mean()
print(clazz, math.log(1/scale))
params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'binary_logloss'},
                'num_leaves': num_leaves.get(clazz, 32),
                'max_depth': max_depth.get(clazz, 6),
                'learning_rate': learning_rate.get(clazz, 0.009),
                'feature_fraction':  feature_fraction.get(clazz, 0.6),
                'bagging_fraction': bagging_fraction.get(clazz, 0.8),
                'bagging_freq': bagging_freq.get(clazz, 5),
                'scale_pos_weight':scale_pos_weight.get(clazz, 1),
                'lambda_l1':lambda_l1.get(clazEarly stopping, best iteration is:
[907]	training's binary_logloss: 0.0264165	valid_1's binary_logloss: 0.0319882
empty 0.030919406864945046ples.get(clazz, 25),

}


gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=3000,
                    valid_sets=[lgb_train, lgb_val],
                    early_stopping_rounds=20)
gbm.save_model("lgbs_v8/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

X_test = test_dataset[columns]
y_test = test_dataset[clazz]

print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
# [784]	training's binary_logloss: 0.0256401	valid_1's binary_logloss: 0.0322288
# empty 0.0310637396503383

# Early stopping, best iteration is:
# [907]	training's binary_logloss: 0.0264165	valid_1's binary_logloss: 0.0319882
# empty 0.030919406864945046

empty 0.24382992559749844
[1]	training's binary_logloss: 0.514748	valid_1's binary_logloss: 0.492686
Training until validation scores don't improve for 20 rounds
[2]	training's binary_logloss: 0.507442	valid_1's binary_logloss: 0.485814
[3]	training's binary_logloss: 0.500358	valid_1's binary_logloss: 0.479141
[4]	training's binary_logloss: 0.493468	valid_1's binary_logloss: 0.472643
[5]	training's binary_logloss: 0.486793	valid_1's binary_logloss: 0.46632
[6]	training's binary_logloss: 0.480282	valid_1's binary_logloss: 0.460179
[7]	training's binary_logloss: 0.473954	valid_1's binary_logloss: 0.454172
[8]	training's binary_logloss: 0.467801	valid_1's binary_logloss: 0.448345
[9]	training's binary_logloss: 0.461817	valid_1's binary_logloss: 0.442649
[10]	training's binary_logloss: 0.455946	valid_1's binary_logloss: 0.437112
[11]	training's binary_logloss: 0.450257	valid_1's binary_logloss: 0.431706
[12]	training's binary_logloss: 0.444672	valid_1's binary_logloss: 0.426412
[13]	traini

[108]	training's binary_logloss: 0.180178	valid_1's binary_logloss: 0.174609
[109]	training's binary_logloss: 0.178779	valid_1's binary_logloss: 0.173273
[110]	training's binary_logloss: 0.177393	valid_1's binary_logloss: 0.171956
[111]	training's binary_logloss: 0.176019	valid_1's binary_logloss: 0.170643
[112]	training's binary_logloss: 0.174663	valid_1's binary_logloss: 0.169352
[113]	training's binary_logloss: 0.173331	valid_1's binary_logloss: 0.168074
[114]	training's binary_logloss: 0.172057	valid_1's binary_logloss: 0.166855
[115]	training's binary_logloss: 0.170742	valid_1's binary_logloss: 0.165594
[116]	training's binary_logloss: 0.169446	valid_1's binary_logloss: 0.164349
[117]	training's binary_logloss: 0.168158	valid_1's binary_logloss: 0.163118
[118]	training's binary_logloss: 0.166883	valid_1's binary_logloss: 0.161897
[119]	training's binary_logloss: 0.165623	valid_1's binary_logloss: 0.160703
[120]	training's binary_logloss: 0.164376	valid_1's binary_logloss: 0.159515

[215]	training's binary_logloss: 0.0878663	valid_1's binary_logloss: 0.0863419
[216]	training's binary_logloss: 0.0873687	valid_1's binary_logloss: 0.0858574
[217]	training's binary_logloss: 0.0868762	valid_1's binary_logloss: 0.0853857
[218]	training's binary_logloss: 0.0863876	valid_1's binary_logloss: 0.0849135
[219]	training's binary_logloss: 0.0859026	valid_1's binary_logloss: 0.0844466
[220]	training's binary_logloss: 0.0854175	valid_1's binary_logloss: 0.0839937
[221]	training's binary_logloss: 0.0849449	valid_1's binary_logloss: 0.0835466
[222]	training's binary_logloss: 0.0844691	valid_1's binary_logloss: 0.0830962
[223]	training's binary_logloss: 0.083996	valid_1's binary_logloss: 0.0826487
[224]	training's binary_logloss: 0.0835347	valid_1's binary_logloss: 0.0822082
[225]	training's binary_logloss: 0.0830709	valid_1's binary_logloss: 0.0817734
[226]	training's binary_logloss: 0.0826077	valid_1's binary_logloss: 0.0813367
[227]	training's binary_logloss: 0.082152	valid_1's b

[319]	training's binary_logloss: 0.05343	valid_1's binary_logloss: 0.0539121
[320]	training's binary_logloss: 0.0532216	valid_1's binary_logloss: 0.0537243
[321]	training's binary_logloss: 0.0530191	valid_1's binary_logloss: 0.0535366
[322]	training's binary_logloss: 0.0528278	valid_1's binary_logloss: 0.0533501
[323]	training's binary_logloss: 0.0526264	valid_1's binary_logloss: 0.0531635
[324]	training's binary_logloss: 0.0524259	valid_1's binary_logloss: 0.0529832
[325]	training's binary_logloss: 0.0522265	valid_1's binary_logloss: 0.0528051
[326]	training's binary_logloss: 0.0520276	valid_1's binary_logloss: 0.0526223
[327]	training's binary_logloss: 0.0518318	valid_1's binary_logloss: 0.0524452
[328]	training's binary_logloss: 0.0516383	valid_1's binary_logloss: 0.0522723
[329]	training's binary_logloss: 0.051446	valid_1's binary_logloss: 0.0520975
[330]	training's binary_logloss: 0.0512567	valid_1's binary_logloss: 0.051926
[331]	training's binary_logloss: 0.0510682	valid_1's bin

[425]	training's binary_logloss: 0.0387044	valid_1's binary_logloss: 0.0406215
[426]	training's binary_logloss: 0.0386163	valid_1's binary_logloss: 0.0405442
[427]	training's binary_logloss: 0.0385284	valid_1's binary_logloss: 0.0404671
[428]	training's binary_logloss: 0.0384416	valid_1's binary_logloss: 0.0403959
[429]	training's binary_logloss: 0.038356	valid_1's binary_logloss: 0.040324
[430]	training's binary_logloss: 0.0382706	valid_1's binary_logloss: 0.0402508
[431]	training's binary_logloss: 0.0381858	valid_1's binary_logloss: 0.0401771
[432]	training's binary_logloss: 0.0381009	valid_1's binary_logloss: 0.0401067
[433]	training's binary_logloss: 0.0380176	valid_1's binary_logloss: 0.0400362
[434]	training's binary_logloss: 0.0379353	valid_1's binary_logloss: 0.0399641
[435]	training's binary_logloss: 0.0378533	valid_1's binary_logloss: 0.0398916
[436]	training's binary_logloss: 0.0377702	valid_1's binary_logloss: 0.0398216
[437]	training's binary_logloss: 0.0376914	valid_1's b

[530]	training's binary_logloss: 0.0323583	valid_1's binary_logloss: 0.0353093
[531]	training's binary_logloss: 0.0323177	valid_1's binary_logloss: 0.0352772
[532]	training's binary_logloss: 0.0322784	valid_1's binary_logloss: 0.0352445
[533]	training's binary_logloss: 0.0322394	valid_1's binary_logloss: 0.0352113
[534]	training's binary_logloss: 0.0322004	valid_1's binary_logloss: 0.0351799
[535]	training's binary_logloss: 0.0321617	valid_1's binary_logloss: 0.0351471
[536]	training's binary_logloss: 0.0321238	valid_1's binary_logloss: 0.0351141
[537]	training's binary_logloss: 0.0320859	valid_1's binary_logloss: 0.0350832
[538]	training's binary_logloss: 0.0320479	valid_1's binary_logloss: 0.0350516
[539]	training's binary_logloss: 0.0320109	valid_1's binary_logloss: 0.0350243
[540]	training's binary_logloss: 0.0319743	valid_1's binary_logloss: 0.0349947
[541]	training's binary_logloss: 0.0319378	valid_1's binary_logloss: 0.034968
[542]	training's binary_logloss: 0.0319022	valid_1's 

[635]	training's binary_logloss: 0.0293792	valid_1's binary_logloss: 0.0331612
[636]	training's binary_logloss: 0.0293596	valid_1's binary_logloss: 0.0331489
[637]	training's binary_logloss: 0.0293402	valid_1's binary_logloss: 0.0331369
[638]	training's binary_logloss: 0.029321	valid_1's binary_logloss: 0.0331261
[639]	training's binary_logloss: 0.0293013	valid_1's binary_logloss: 0.0331128
[640]	training's binary_logloss: 0.0292812	valid_1's binary_logloss: 0.0330998
[641]	training's binary_logloss: 0.0292635	valid_1's binary_logloss: 0.0330876
[642]	training's binary_logloss: 0.0292453	valid_1's binary_logloss: 0.0330731
[643]	training's binary_logloss: 0.029227	valid_1's binary_logloss: 0.0330622
[644]	training's binary_logloss: 0.0292084	valid_1's binary_logloss: 0.0330491
[645]	training's binary_logloss: 0.0291905	valid_1's binary_logloss: 0.0330361
[646]	training's binary_logloss: 0.029173	valid_1's binary_logloss: 0.0330277
[647]	training's binary_logloss: 0.0291556	valid_1's bi

[740]	training's binary_logloss: 0.0278176	valid_1's binary_logloss: 0.0323524
[741]	training's binary_logloss: 0.0278054	valid_1's binary_logloss: 0.0323486
[742]	training's binary_logloss: 0.027793	valid_1's binary_logloss: 0.0323411
[743]	training's binary_logloss: 0.0277817	valid_1's binary_logloss: 0.0323372
[744]	training's binary_logloss: 0.0277705	valid_1's binary_logloss: 0.0323319
[745]	training's binary_logloss: 0.0277607	valid_1's binary_logloss: 0.0323257
[746]	training's binary_logloss: 0.0277475	valid_1's binary_logloss: 0.0323227
[747]	training's binary_logloss: 0.0277371	valid_1's binary_logloss: 0.0323197
[748]	training's binary_logloss: 0.0277266	valid_1's binary_logloss: 0.0323156
[749]	training's binary_logloss: 0.0277158	valid_1's binary_logloss: 0.0323136
[750]	training's binary_logloss: 0.0277041	valid_1's binary_logloss: 0.0323111
[751]	training's binary_logloss: 0.0276936	valid_1's binary_logloss: 0.0323051
[752]	training's binary_logloss: 0.0276825	valid_1's 

[845]	training's binary_logloss: 0.0268533	valid_1's binary_logloss: 0.0320396
[846]	training's binary_logloss: 0.0268459	valid_1's binary_logloss: 0.0320379
[847]	training's binary_logloss: 0.0268384	valid_1's binary_logloss: 0.0320347
[848]	training's binary_logloss: 0.0268297	valid_1's binary_logloss: 0.0320319
[849]	training's binary_logloss: 0.0268226	valid_1's binary_logloss: 0.0320266
[850]	training's binary_logloss: 0.0268151	valid_1's binary_logloss: 0.0320263
[851]	training's binary_logloss: 0.0268075	valid_1's binary_logloss: 0.0320252
[852]	training's binary_logloss: 0.0267997	valid_1's binary_logloss: 0.0320292
[853]	training's binary_logloss: 0.026793	valid_1's binary_logloss: 0.0320276
[854]	training's binary_logloss: 0.0267857	valid_1's binary_logloss: 0.0320264
[855]	training's binary_logloss: 0.0267787	valid_1's binary_logloss: 0.0320243
[856]	training's binary_logloss: 0.0267735	valid_1's binary_logloss: 0.0320243
[857]	training's binary_logloss: 0.0267661	valid_1's 

In [113]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance(),X_test.columns)), columns=['Value','Feature'])
feature_imp[-100:]


,Value,Feature
1384,17,time_wildebeest
1385,18,c2_269
1386,18,c2_272
1387,18,c3_340
1388,18,c3_984
1389,18,mean_combines_baboon
1390,18,mean_with_combines_imgs_baboon
1391,18,median_img3_lionmale
1392,18,pred_model_480_360_as480_img3_otherbird
1393,18,pred_model_512_384_v2_as512_img2_hippopotamus


In [38]:
clazzs = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird']


        
for clazz in clazzs:
    
#     feature_classes=feature_classes_d.get(clazz, list(train_labels_columns))

    columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']



    X_train = train_dataset[columns]
    y_train = train_dataset[clazz]
    X_val = val_dataset[columns]
    y_val = val_dataset[clazz]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'binary_logloss'},
                'num_leaves': num_leaves.get(clazz, 32),
                'max_depth': max_depth.get(clazz, 6),
                'learning_rate': learning_rate.get(clazz, 0.009),
                'feature_fraction':  feature_fraction.get(clazz, 0.6),
                'bagging_fraction': bagging_fraction.get(clazz, 0.8),
                'bagging_freq': bagging_freq.get(clazz, 5),
                'scale_pos_weight':scale_pos_weight.get(clazz, 1),
                'lambda_l1':lambda_l1.get(clazz, 0.2),
                'lambda_l2':lambda_l2.get(clazz, 0.2),
                'min_child_samples':min_child_samples.get(clazz, 25),

    }

    gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=3000,
                        valid_sets=[lgb_train, lgb_val],
                        early_stopping_rounds=20)
    gbm.save_model("lgbs_v8/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

    X_test = test_dataset[columns]
    y_test = test_dataset[clazz]
    
    print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
    gc.collect()
    

[1]	training's binary_logloss: 0.235993	valid_1's binary_logloss: 0.287224
Training until validation scores don't improve for 20 rounds
[2]	training's binary_logloss: 0.228596	valid_1's binary_logloss: 0.277749
[3]	training's binary_logloss: 0.221949	valid_1's binary_logloss: 0.269267
[4]	training's binary_logloss: 0.215914	valid_1's binary_logloss: 0.26157
[5]	training's binary_logloss: 0.210386	valid_1's binary_logloss: 0.254532
[6]	training's binary_logloss: 0.205276	valid_1's binary_logloss: 0.248029
[7]	training's binary_logloss: 0.200521	valid_1's binary_logloss: 0.242005
[8]	training's binary_logloss: 0.196078	valid_1's binary_logloss: 0.236386
[9]	training's binary_logloss: 0.191907	valid_1's binary_logloss: 0.23112
[10]	training's binary_logloss: 0.187976	valid_1's binary_logloss: 0.226168
[11]	training's binary_logloss: 0.184257	valid_1's binary_logloss: 0.221478
[12]	training's binary_logloss: 0.180723	valid_1's binary_logloss: 0.217041
[13]	training's binary_logloss: 0.1773

[108]	training's binary_logloss: 0.0618182	valid_1's binary_logloss: 0.0732741
[109]	training's binary_logloss: 0.0613079	valid_1's binary_logloss: 0.072681
[110]	training's binary_logloss: 0.0608037	valid_1's binary_logloss: 0.0720984
[111]	training's binary_logloss: 0.0603055	valid_1's binary_logloss: 0.0715244
[112]	training's binary_logloss: 0.0598128	valid_1's binary_logloss: 0.0709606
[113]	training's binary_logloss: 0.0593267	valid_1's binary_logloss: 0.0703988
[114]	training's binary_logloss: 0.058846	valid_1's binary_logloss: 0.0698443
[115]	training's binary_logloss: 0.0583707	valid_1's binary_logloss: 0.0692973
[116]	training's binary_logloss: 0.0579012	valid_1's binary_logloss: 0.0687559
[117]	training's binary_logloss: 0.0574374	valid_1's binary_logloss: 0.068222
[118]	training's binary_logloss: 0.0569785	valid_1's binary_logloss: 0.067695
[119]	training's binary_logloss: 0.0565252	valid_1's binary_logloss: 0.067174
[120]	training's binary_logloss: 0.0560789	valid_1's bina

[212]	training's binary_logloss: 0.0301488	valid_1's binary_logloss: 0.0374354
[213]	training's binary_logloss: 0.0299799	valid_1's binary_logloss: 0.0372493
[214]	training's binary_logloss: 0.0298128	valid_1's binary_logloss: 0.0370644
[215]	training's binary_logloss: 0.0296474	valid_1's binary_logloss: 0.0368819
[216]	training's binary_logloss: 0.0294827	valid_1's binary_logloss: 0.0367001
[217]	training's binary_logloss: 0.0293208	valid_1's binary_logloss: 0.036521
[218]	training's binary_logloss: 0.0291603	valid_1's binary_logloss: 0.0363435
[219]	training's binary_logloss: 0.0290012	valid_1's binary_logloss: 0.0361659
[220]	training's binary_logloss: 0.0288434	valid_1's binary_logloss: 0.0359941
[221]	training's binary_logloss: 0.0286863	valid_1's binary_logloss: 0.0358212
[222]	training's binary_logloss: 0.0285311	valid_1's binary_logloss: 0.0356516
[223]	training's binary_logloss: 0.0283782	valid_1's binary_logloss: 0.0354826
[224]	training's binary_logloss: 0.0282261	valid_1's 

[316]	training's binary_logloss: 0.0188058	valid_1's binary_logloss: 0.0252633
[317]	training's binary_logloss: 0.01874	valid_1's binary_logloss: 0.0251946
[318]	training's binary_logloss: 0.0186742	valid_1's binary_logloss: 0.0251282
[319]	training's binary_logloss: 0.0186094	valid_1's binary_logloss: 0.0250637
[320]	training's binary_logloss: 0.0185453	valid_1's binary_logloss: 0.024998
[321]	training's binary_logloss: 0.0184825	valid_1's binary_logloss: 0.024934
[322]	training's binary_logloss: 0.0184193	valid_1's binary_logloss: 0.0248688
[323]	training's binary_logloss: 0.0183573	valid_1's binary_logloss: 0.024806
[324]	training's binary_logloss: 0.0182949	valid_1's binary_logloss: 0.0247417
[325]	training's binary_logloss: 0.0182331	valid_1's binary_logloss: 0.0246791
[326]	training's binary_logloss: 0.0181728	valid_1's binary_logloss: 0.0246167
[327]	training's binary_logloss: 0.0181125	valid_1's binary_logloss: 0.0245544
[328]	training's binary_logloss: 0.0180529	valid_1's bina

[421]	training's binary_logloss: 0.0142021	valid_1's binary_logloss: 0.020783
[422]	training's binary_logloss: 0.0141752	valid_1's binary_logloss: 0.0207557
[423]	training's binary_logloss: 0.0141483	valid_1's binary_logloss: 0.0207297
[424]	training's binary_logloss: 0.0141213	valid_1's binary_logloss: 0.0207041
[425]	training's binary_logloss: 0.0140954	valid_1's binary_logloss: 0.0206798
[426]	training's binary_logloss: 0.0140686	valid_1's binary_logloss: 0.020656
[427]	training's binary_logloss: 0.0140419	valid_1's binary_logloss: 0.0206294
[428]	training's binary_logloss: 0.0140158	valid_1's binary_logloss: 0.0206054
[429]	training's binary_logloss: 0.0139897	valid_1's binary_logloss: 0.0205808
[430]	training's binary_logloss: 0.0139657	valid_1's binary_logloss: 0.0205589
[431]	training's binary_logloss: 0.01394	valid_1's binary_logloss: 0.0205367
[432]	training's binary_logloss: 0.0139145	valid_1's binary_logloss: 0.0205133
[433]	training's binary_logloss: 0.0138886	valid_1's bin

...

[330]	training's binary_logloss: 0.00032855	valid_1's binary_logloss: 0.00270085
[331]	training's binary_logloss: 0.000326552	valid_1's binary_logloss: 0.00269784
[332]	training's binary_logloss: 0.000324339	valid_1's binary_logloss: 0.00269591
[333]	training's binary_logloss: 0.000322097	valid_1's binary_logloss: 0.00269397
[334]	training's binary_logloss: 0.000319988	valid_1's binary_logloss: 0.00269181
[335]	training's binary_logloss: 0.000317869	valid_1's binary_logloss: 0.00269047
[336]	training's binary_logloss: 0.000315743	valid_1's binary_logloss: 0.00268844
[337]	training's binary_logloss: 0.000313582	valid_1's binary_logloss: 0.00268708
[338]	training's binary_logloss: 0.000311608	valid_1's binary_logloss: 0.00268585
[339]	training's binary_logloss: 0.000309628	valid_1's binary_logloss: 0.00268353
[340]	training's binary_logloss: 0.000307751	valid_1's binary_logloss: 0.0026822
[341]	training's binary_logloss: 0.000305898	valid_1's binary_logloss: 0.00268123
[342]	training's b

[431]	training's binary_logloss: 0.000177874	valid_1's binary_logloss: 0.00264176
[432]	training's binary_logloss: 0.000176765	valid_1's binary_logloss: 0.00264228
[433]	training's binary_logloss: 0.000175868	valid_1's binary_logloss: 0.00264363
[434]	training's binary_logloss: 0.000174901	valid_1's binary_logloss: 0.0026436
[435]	training's binary_logloss: 0.000173845	valid_1's binary_logloss: 0.00264385
[436]	training's binary_logloss: 0.000172883	valid_1's binary_logloss: 0.00264476
Early stopping, best iteration is:
[416]	training's binary_logloss: 0.000193818	valid_1's binary_logloss: 0.00263796
guineafowl 0.0027139000064911583
[1]	training's binary_logloss: 0.0199212	valid_1's binary_logloss: 0.0219326
Training until validation scores don't improve for 20 rounds
[2]	training's binary_logloss: 0.0189501	valid_1's binary_logloss: 0.020882
[3]	training's binary_logloss: 0.0181914	valid_1's binary_logloss: 0.0201277
[4]	training's binary_logloss: 0.0176229	valid_1's binary_logloss: 0

[97]	training's binary_logloss: 0.00868805	valid_1's binary_logloss: 0.0117588
[98]	training's binary_logloss: 0.00865645	valid_1's binary_logloss: 0.011739
[99]	training's binary_logloss: 0.00862672	valid_1's binary_logloss: 0.0117206
[100]	training's binary_logloss: 0.00859712	valid_1's binary_logloss: 0.011703
[101]	training's binary_logloss: 0.00856974	valid_1's binary_logloss: 0.011686
[102]	training's binary_logloss: 0.00854036	valid_1's binary_logloss: 0.0116692
[103]	training's binary_logloss: 0.00851321	valid_1's binary_logloss: 0.0116513
[104]	training's binary_logloss: 0.0084862	valid_1's binary_logloss: 0.0116341
[105]	training's binary_logloss: 0.008461	valid_1's binary_logloss: 0.0116145
[106]	training's binary_logloss: 0.00843455	valid_1's binary_logloss: 0.0115992
[107]	training's binary_logloss: 0.00840857	valid_1's binary_logloss: 0.0115828
[108]	training's binary_logloss: 0.00838207	valid_1's binary_logloss: 0.0115695
[109]	training's binary_logloss: 0.00835576	valid

[200]	training's binary_logloss: 0.00682991	valid_1's binary_logloss: 0.0108482
[201]	training's binary_logloss: 0.00681739	valid_1's binary_logloss: 0.010847
[202]	training's binary_logloss: 0.00680689	valid_1's binary_logloss: 0.0108427
[203]	training's binary_logloss: 0.00679724	valid_1's binary_logloss: 0.0108391
[204]	training's binary_logloss: 0.00678671	valid_1's binary_logloss: 0.0108367
[205]	training's binary_logloss: 0.00677745	valid_1's binary_logloss: 0.0108327
[206]	training's binary_logloss: 0.00676729	valid_1's binary_logloss: 0.0108279
[207]	training's binary_logloss: 0.00675739	valid_1's binary_logloss: 0.0108245
[208]	training's binary_logloss: 0.0067475	valid_1's binary_logloss: 0.0108216
[209]	training's binary_logloss: 0.00673742	valid_1's binary_logloss: 0.0108211
[210]	training's binary_logloss: 0.0067264	valid_1's binary_logloss: 0.0108202
[211]	training's binary_logloss: 0.0067153	valid_1's binary_logloss: 0.0108172
[212]	training's binary_logloss: 0.00670493	

[303]	training's binary_logloss: 0.00597918	valid_1's binary_logloss: 0.0107003
[304]	training's binary_logloss: 0.00597209	valid_1's binary_logloss: 0.0106991
[305]	training's binary_logloss: 0.00596571	valid_1's binary_logloss: 0.0106986
[306]	training's binary_logloss: 0.00595903	valid_1's binary_logloss: 0.0106958
[307]	training's binary_logloss: 0.00595227	valid_1's binary_logloss: 0.0106947
[308]	training's binary_logloss: 0.00594592	valid_1's binary_logloss: 0.0106938
[309]	training's binary_logloss: 0.00593935	valid_1's binary_logloss: 0.0106937
[310]	training's binary_logloss: 0.00593246	valid_1's binary_logloss: 0.0106944
[311]	training's binary_logloss: 0.0059263	valid_1's binary_logloss: 0.0106933
[312]	training's binary_logloss: 0.00592047	valid_1's binary_logloss: 0.0106928
[313]	training's binary_logloss: 0.0059145	valid_1's binary_logloss: 0.0106937
[314]	training's binary_logloss: 0.00590938	valid_1's binary_logloss: 0.0106929
[315]	training's binary_logloss: 0.0059033

In [27]:
gc.collect()

138